In [ ]:
!pip install transformers
!pip install --upgrade xlrd

In [ ]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from sklearn.utils import resample
from transformers import BertForSequenceClassification
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
letters=pd.read_excel('./drive/MyDrive/Colab Notebooks/MailClassification/prediction6000.xls')
letters.head(3)

,Unnamed: 0,subject,address,domain,main_text,type,verified,val2
0,0,FW: Заявка № БС 21307 на устранение ошибки в «...,ios,cbr.ru,Уважаемые коллеги!\r\n\r\nПрошу оперативно соо...,2.0,2.0,NaN
1,1,RE: Заявка № БС 21315 на устранение ошибки в «...,kai,cbr.ru,Прошу уточнить:\r\n\r\n1. Велась ли ране...,2.0,2.0,NaN
2,2,RE: Заявка № БС 21316 на устранение ошибки в «...,dda,cbr.ru,"Уважаемые коллеги!\r\n\r\nПрошу уточнить, что ...",2.0,2.0,NaN


In [ ]:
letters = letters[['subject','address','domain','main_text', 'verified']]
letters.head(3)             

,subject,address,domain,main_text,verified
0,FW: Заявка № БС 21307 на устранение ошибки в «...,ios,cbr.ru,Уважаемые коллеги!\r\n\r\nПрошу оперативно соо...,2.0
1,RE: Заявка № БС 21315 на устранение ошибки в «...,kai,cbr.ru,Прошу уточнить:\r\n\r\n1. Велась ли ране...,2.0
2,RE: Заявка № БС 21316 на устранение ошибки в «...,dda,cbr.ru,"Уважаемые коллеги!\r\n\r\nПрошу уточнить, что ...",2.0


In [ ]:
#Категории писем
#1. Требует регистрации заявки
#2. Сообщение (доведение информации, не требующее реагирования в рамках сопровождения)
#3. Уведомление о подтверждении РС
#4. Уведомление об установке версии
#5. Запросы на дежурство в выходной день
letters= letters.dropna()
letters['verified'].value_counts()

2.0    5561
1.0     395
3.0      27
4.0      19
5.0       2
Name: verified, dtype: int64

In [ ]:
#уберем пока 5 категорию, т.к. запросов на работу только 2 и смысла в них пока нет
letters=letters[letters['verified']!=5]
letters['verified']=letters['verified'].astype('int')

In [ ]:
#для начала попробуем забаллансить классы
len(letters[letters['verified']==2]) /letters['verified'].value_counts() 

2      1.000000
1     14.078481
3    205.962963
4    292.684211
Name: verified, dtype: float64

In [ ]:
max_size = len(letters[letters['verified']==2]) 
up_letters =pd.concat([resample(letters[letters['verified']==x], replace=True, n_samples=max_size, random_state=240122) for x in range(1,5)])

In [ ]:
up_letters['verified'].value_counts() 

1    5561
2    5561
3    5561
4    5561
Name: verified, dtype: int64

In [ ]:
def _clear_text_cyr_lat(text): #чистка от пунктуации и пр.
            try: 
                text=re.sub(r'[^а-яА-ЯёЁa-zA-Z]',' ', text)
                return ' '.join(text.split())  
            except TypeError:    
                print(type(text))
                return ''

In [ ]:
up_letters['subject']=up_letters['subject'].apply(lambda x: _clear_text_cyr_lat(x))
up_letters['main_text']=up_letters['main_text'].apply(lambda x: _clear_text_cyr_lat(x))

In [ ]:
df = up_letters[['main_text','verified','subject','domain', 'address']].reset_index(drop=True).copy()
df.head()

,main_text,verified,subject,domain,address
0,Уважаемые коллеги Зарегистрирован инцидент в А...,1,Инцидент в АС ДНДО,cbr.ru,RuchnovSV
1,Коллеги прошу запланировать доработку С уважен...,1,Доработка АС ДНДО,itlimon.ru,cherepanov
2,Уважаемые коллеги От пользователя АС ДНДО Обра...,1,АС ДНДО INC,cbr.ru,sadovnikovsa
3,Пересылаю инцидент от пользователя ДЕНИСОВ ЕВГ...,1,FW Инцидент INC назначен вам Приоритет Средний...,cbr.ru,dea
4,Добрый день Не загрузилась форма из Татарстана...,1,Ошибка при загрузке формы из Татарстана,cbr.ru,dea


In [ ]:
df['verified'].value_counts()

1    5561
2    5561
3    5561
4    5561
Name: verified, dtype: int64

In [ ]:
possible_labels = df['verified'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{1: 0, 2: 1, 3: 2, 4: 3}

In [ ]:
df['label'] = df['verified'].replace(label_dict)

In [ ]:
df.shape

(22244, 6)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df['label'].values, 
                                                  test_size=0.15, 
                                                  random_state=240122, 
                                                  stratify=df['label'].values)

In [ ]:
print(X_train.shape)
print(X_val.shape)

(18907,)
(3337,)


In [ ]:
#разметка данных на трейн и тест
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

main_text  verified  subject  domain  address
label data_type                                               
0     train           4727      4727     4727    4727     4727
      val              834       834      834     834      834
1     train           4726      4726     4726    4726     4726
      val              835       835      835     835      835
2     train           4727      4727     4727    4727     4727
      val              834       834      834     834      834
3     train           4727      4727     4727    4727     4727
      val              834       834      834     834      834

In [ ]:
df['text']=df['domain']+' '+df['address']+' '+df['subject']+df['main_text']
df.head(2)

,main_text,verified,subject,domain,address,label,data_type,text
0,Уважаемые коллеги Зарегистрирован инцидент в А...,1,Инцидент в АС ДНДО,cbr.ru,RuchnovSV,0,train,cbr.ru RuchnovSV Инцидент в АС ДНДОУважаемые к...
1,Коллеги прошу запланировать доработку С уважен...,1,Доработка АС ДНДО,itlimon.ru,cherepanov,0,train,itlimon.ru cherepanov Доработка АС ДНДОКоллеги...


In [ ]:
vocab  = 'bert-base-uncased' #'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(vocab, 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(18907, 3337)

In [ ]:
model = BertForSequenceClassification.from_pretrained(vocab,
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 240122
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(),'./drive/MyDrive/Colab Notebooks/MailClassification/finetuned_BERT_fullinfo_epoch_{:}.model'.format(epoch))
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1182 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.28072719570456545
Validation loss: 0.11429640369932056
F1 Score (Weighted): 0.9700836948337518


Epoch 2:   0%|          | 0/1182 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.056702261238940795
Validation loss: 0.031658448653424306
F1 Score (Weighted): 0.993397515285701


...


Epoch 8
Training loss: 0.003833632308500341
Validation loss: 0.014111849167024586
F1 Score (Weighted): 0.9979004905008617


Epoch 9:   0%|          | 0/1182 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.0013657085551077015
Validation loss: 0.027883812226744985
F1 Score (Weighted): 0.9964002304969695


Epoch 10:   0%|          | 0/1182 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.0008130399443922068
Validation loss: 0.017701123313639128
F1 Score (Weighted): 0.9979022758328938


In [ ]:
model = BertForSequenceClassification.from_pretrained(vocab,
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/Colab Notebooks/MailClassification/finetuned_BERT_fullinfo_epoch_10.model')) #,map_location=torch.device('cpu')

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 1
Accuracy: 834/834

Class: 2
Accuracy: 828/835

Class: 3
Accuracy: 834/834

Class: 4
Accuracy: 834/834

